<a href="https://colab.research.google.com/github/prakashgypsii/RAGSystem/blob/main/RAGSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q langchain faiss-cpu sentence-transformers transformers accelerate

In [6]:
!pip install -U langchain-community

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

In [10]:
from google.colab import files

uploaded = files.upload()
pdf_name = list(uploaded.keys())[0]
print("Uploaded file:", pdf_name)
pdf_path = f"/content/{pdf_name}"

Saving Sample.pdf to Sample (1).pdf
Uploaded file: Sample (1).pdf


In [12]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 20.1 MB/s eta 0:00:00


In [13]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(pdf_path)
docs = loader.load()

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = splitter.split_documents(docs)

In [16]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(documents, embeddings)

<ipython-input-16-6362b28e0315>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline

llm_pipe = pipeline("text-generation",
                    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                    device=-1,  # use CPU
                    max_new_tokens=100,
                    temperature=0.5)

llm = HuggingFacePipeline(pipeline=llm_pipe)
qa = RetrievalQA.from_chain_type(llm=llm, retriever=db.as_retriever())

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-17-252711f188c9>:11: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipe)


In [19]:
query = "what is the dress code for men associates?"
result = qa.run(query)
print("Answer:", result)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Dress CodeWOMEN ASSOCIATES: Salwar Khameez / Saree with sandals / shoes  Business Wear (Monday to Friday): • Sarees • Salwar / Long Kurta / Churidhar kameez• Sandals or closed shoes Business Casuals (Saturday / Holidays) : • Sarees • Salwar / Long Kurta / Churidhar kameez• Denim pants with t-shirt / kurta• Sandals or closed shoes MEN ASSOCIATES : Formal shirt with full length formal pant and formal shoes.Business Formals (Monday to Friday):Full sleeved formal shirtsFormal Pants Black or brown

formal shirtsFormal Pants Black or brown formal shoes with socks. Business Casuals (Saturday / Holiday):Casual shirts / t-shirts / Polo t-shirtsCasual pants / denim pantsCasual shoes, socks at all timesCasual dressing to be avoided incase of customer meetings.Office ID Cards:Associates are required to wear their ID car